In [1]:
import random
import json
import os
import sys

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image

from resnet_pytorch import ResNet
import wandb
import uuid

# load .env file
from dotenv import load_dotenv
load_dotenv()

sys.path.insert(0, '../')
from data_loader import get_data_to_load, split_json_and_image_files, load_json_files, load_image_files, load_json_file, load_image_file

### Loading data

In [2]:
# set number of files to load
NUMBER_OF_FILES = 10000

# get list with local data and file paths
list_files = get_data_to_load(loading_file='../3_data_preparation/04_data_cleaning/updated_data_list', file_location='../3_data_preparation/01_enriching/.data', image_file_location='../1_data_collection/.data', allow_new_file_creation=False, from_remote_only=True, download_link='env', limit=NUMBER_OF_FILES, shuffle_seed=42, allow_file_location_env=True, allow_json_file_location_env=True, allow_image_file_location_env=True)

json_files, image_files = split_json_and_image_files(list_files)
paired_files = list(zip(json_files, image_files))

All local files: 403361
Relevant files: 403361
Limited files: 30000


### example linus

In [3]:
class CustomImageNameDataset(Dataset):
    def __init__(self, image_paths, json_paths, transform=None):
        self.image_paths = image_paths
        self.json_paths = json_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        return self.image_paths[idx], self.json_paths[idx]

# Define transformations
transform = transforms.Compose([
        transforms.Resize((50, 50)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

In [4]:
assert len(image_files) == len(json_files), "Mismatch in number of images and labels"

file_name_dataset = CustomImageNameDataset(image_files, json_files, transform=transform)
file_name_loader = DataLoader(file_name_dataset, batch_size=64, shuffle=True, num_workers=0)

In [5]:
file_name_loader.dataset.image_paths[0]

'/Volumes/LinUSB/combined/geoguessr_location_singleplayer_01DXbeVAxrsJR1Zu_1.png'

In [6]:
for temp_image_files, temp_label_files in file_name_loader:
    images = load_image_files(temp_image_files)
    labels = load_json_files(temp_label_files)
    countries = [item['country_name'] for item in labels]
    coordinates = [item['coordinates'] for item in labels]
    transformed_images = []
    for image in images:
      transformed_images.append(transform(image))
    break  # After the first batch, exit the loop
print("Images:", len(images))
print("Labels:", len(labels))

Images: 64
Labels: 64


### example lukas

In [7]:
class CustomImageDataset(Dataset):
    def __init__(self, images, coordinates, countries):
        self.images = images
        self.coordinates = coordinates
        self.countries = countries
        self.country_to_index = {country: idx for idx, country in enumerate(set(countries))}

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        country_index = self.country_to_index[self.countries[idx]]
        coordinates = torch.tensor(self.coordinates, dtype=torch.float32)

        return image, coordinates, country_index

class ImageDataHandler:
    def __init__(self, image_paths, json_paths, batch_size=64, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
        self.batch_size = batch_size
        
        print(len(image_paths))
      
        file_name_dataset = CustomImageNameDataset(image_paths, json_paths, transform=transform)
        file_name_loader = DataLoader(file_name_dataset, batch_size=batch_size, shuffle=False)
        
        self.images = []
        self.countries = []
        self.coordinates = []

        for batch_image_files, batch_label_files in file_name_loader:
            images = load_image_files(batch_image_files)
            labels = load_json_files(batch_label_files)
            self.countries.extend([item['country_name'] for item in labels])
            self.coordinates.extend([item['coordinates'] for item in labels])
            for image in images:
              self.images.append(transform(image))
        
        # Initialize datasets and loaders
        self.train_loader, self.val_loader, self.test_loader = self.create_loaders(train_ratio, val_ratio, test_ratio)

    def create_loaders(self, train_ratio, val_ratio, test_ratio):
        assert train_ratio + val_ratio + test_ratio - 1 <= 0.001, "Ratios should sum to 1"
        
        combined = list(zip(self.images, self.coordinates, self.countries))
        random.shuffle(combined)
        total_count = len(combined)
        train_end = int(train_ratio * total_count)
        val_end = train_end + int(val_ratio * total_count)

        train_data = combined[:train_end]
        val_data = combined[train_end:val_end]
        test_data = combined[val_end:]
        
        # Create train, val- and test datasets
        train_dataset = CustomImageDataset(*zip(*train_data))
        val_dataset = CustomImageDataset(*zip(*val_data))
        test_dataset = CustomImageDataset(*zip(*test_data))

        # Create train, val- and test dataloaders
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=self.batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)

        return train_loader, val_loader, test_loader

In [8]:
# Creating Dataloasders with the classes
data_handler = ImageDataHandler(image_files, json_files)
train_dataloader = data_handler.train_loader
val_dataloader = data_handler.val_loader
test_dataloader = data_handler.test_loader

print("Number of train batches:", len(train_dataloader))

# Print forst batch as an example, to see the structure
# 7000 images need 59 sec for processing as information
for images, coordinates, country_indices in train_dataloader:
    print("Images batch shape:", images.shape)
    print("Coordinates batch shape:", coordinates.shape)
    print("Country indices:", country_indices.shape)
    break

10000
Number of train batches: 110
Images batch shape: torch.Size([64, 3, 50, 50])
Coordinates batch shape: torch.Size([64, 7000, 2])
Country indices: torch.Size([64])


## Model

In [9]:
# Load the pretrained model
model = ResNet.from_pretrained('resnet18', num_classes=2)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/linus/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 30.2MB/s]


Loaded pretrained weights for resnet18.


In [10]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## Training

In [11]:
# set necessary seeds to make notebook reproducible 
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

In [12]:
def train():
  with wandb.init(reinit=True) as run:
    config = run.config
    set_seed(config.seed)
    wandb.init(project="nlp-project-02", reinit=True, name="lr_" + str(config.learning_rate) + "_opt_" + str(config.optimizer) + "_weightDecay_" + str(config.weight_decay) + str(uuid.uuid4()))
    best_val_loss = float('inf')

    criterion = nn.CrossEntropyLoss()
    model = ResNet.from_pretrained('resnet18', num_classes=2)
    optimizer = optim.SGD(model.parameters(), lr=config.learning_rate, momentum=0.9, weight_decay=config.weight_decay)

    for epoch in range(config.epochs):
        train_loss = 0.0
        model.train()

        for images, coordinates, country_indices in train_dataloader:
            #print("Images shape:", images.shape)
            #print("Coordinates shape:", coordinates.shape)
            #images, coordinates = images.to('cuda'), coordinates.to('cuda')
            optimizer.zero_grad()
            output = model(images)
            print("Output shape:", output.shape)
            loss = criterion(output, coordinates)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_dataloader.dataset)

        val_loss = 0.0
        model.eval()

        with torch.no_grad():
            for images, coordinates, country_indices in val_dataloader:
                #images, coordinates = images.to('cuda'), coordinates.to('cuda')
                output = model(images)
                loss = criterion(output, coordinates)

                val_loss += loss.item() * images.size(0)

        val_loss /= len(val_dataloader.dataset)

        print(f'Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), f"models/resnet18_best_model_checkpoint.pth")

In [13]:
wandb.login()

sweep_config = {
    "name": f"dspro2-basemodel-resnet18",
    "method": "grid",
    "metric": {"goal": "maximize", "name": "eval_accuracy"},
    "parameters": {
        "learning_rate": {"values": [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]},
        "optimizer": {"values": ["SGD"]},
        "weight_decay": {"values": [1e-3]},
        "epochs": {"values": [200]},
        "seed": {"values": [42]}
    },
}

sweep_id = wandb.sweep(sweep=sweep_config, project=f"dspro2-basemodel-resnet18")
wandb.agent(sweep_id, function=train)

NameError: name 'wandb' is not defined